# Airline Dataset Analysis using PySpark.

In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.appName('Flight_df').getOrCreate()

In [3]:
spark

In [4]:
df= spark.read.csv("C:\\Users\\619007\\Documents\\flights.csv",header=True,inferSchema=True)

In [5]:
df.show(2)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [6]:
df.describe()

DataFrame[summary: string, YEAR: string, MONTH: string, DAY: string, DAY_OF_WEEK: string, AIRLINE: string, FLIGHT_NUMBER: string, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: string, DEPARTURE_TIME: string, DEPARTURE_DELAY: string, TAXI_OUT: string, WHEELS_OFF: string, SCHEDULED_TIME: string, ELAPSED_TIME: string, AIR_TIME: string, DISTANCE: string, WHEELS_ON: string, TAXI_IN: string, SCHEDULED_ARRIVAL: string, ARRIVAL_TIME: string, ARRIVAL_DELAY: string, DIVERTED: string, CANCELLED: string, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: string, SECURITY_DELAY: string, AIRLINE_DELAY: string, LATE_AIRCRAFT_DELAY: string, WEATHER_DELAY: string]

In [7]:
df1 = df[['AIRLINE','FLIGHT_NUMBER','ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','DEPARTURE_TIME',
 'DEPARTURE_DELAY','ARRIVAL_TIME']]

In [8]:
df1.show(5)

+-------+-------------+--------------+-------------------+-------------------+--------------+---------------+------------+
|AIRLINE|FLIGHT_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|ARRIVAL_TIME|
+-------+-------------+--------------+-------------------+-------------------+--------------+---------------+------------+
|     AS|           98|           ANC|                SEA|                  5|          2354|            -11|         408|
|     AA|         2336|           LAX|                PBI|                 10|             2|             -8|         741|
|     US|          840|           SFO|                CLT|                 20|            18|             -2|         811|
|     AA|          258|           LAX|                MIA|                 20|            15|             -5|         756|
|     AS|          135|           SEA|                ANC|                 25|            24|             -1|         259|
+-------+-------

In [9]:
df2 = df1[['ORIGIN_AIRPORT']] #'DESTINATION_AIRPORT'

In [10]:
df2.drop_duplicates().count()

315

In [11]:
#Number of Airports
df2.distinct().count()

315

In [12]:
# Number of Trips

df2 = df1.select("FLIGHT_NUMBER") \
            .orderBy("FLIGHT_NUMBER") \
            .groupby("FLIGHT_NUMBER").count() \
            .limit(4)

In [13]:
df2.show()


+-------------+-----+
|FLIGHT_NUMBER|count|
+-------------+-----+
|         1088|  269|
|          148|  349|
|          463|  515|
|          833|  595|
+-------------+-----+



In [14]:
#Determining the longest delay in this dataset----2
#max of Departure Delay
from pyspark.sql.functions import desc,expr,count
from pyspark.sql import functions
df3 = df1.select("Flight_Number","DEPARTURE_DELAY")\
            .where("DEPARTURE_DELAY is not null")\
            .sort(desc("DEPARTURE_DELAY"))\
            .limit(5).show()



+-------------+---------------+
|Flight_Number|DEPARTURE_DELAY|
+-------------+---------------+
|         1322|           1988|
|          224|           1604|
|          270|           1589|
|         1312|           1587|
|         1594|           1557|
+-------------+---------------+



#Determining the number of delayed vs. on-time / early flights



In [15]:
df4 = df1.select(col("FLIGHT_NUMBER"),
                     col("ORIGIN_AIRPORT").alias("Origin"), 
                      col("DESTINATION_AIRPORT").alias("Destination"), 
                      expr("DEPARTURE_DELAY"),
                      expr("DEPARTURE_DELAY < 1 as OnTime_early"),
                      expr("DEPARTURE_DELAY > 1 as Delayed_flight"),
                      expr("ORIGIN_AIRPORT = DESTINATION_AIRPORT as domestic"))

In [16]:
df4.show(4)

+-------------+------+-----------+---------------+------------+--------------+--------+
|FLIGHT_NUMBER|Origin|Destination|DEPARTURE_DELAY|OnTime_early|Delayed_flight|domestic|
+-------------+------+-----------+---------------+------------+--------------+--------+
|           98|   ANC|        SEA|            -11|        true|         false|   false|
|         2336|   LAX|        PBI|             -8|        true|         false|   false|
|          840|   SFO|        CLT|             -2|        true|         false|   false|
|          258|   LAX|        MIA|             -5|        true|         false|   false|
+-------------+------+-----------+---------------+------------+--------------+--------+
only showing top 4 rows



In [18]:
#4.Which flights departing SFO are most likely to have significant delays

df5 = df1.select("FLIGHT_NUMBER","DEPARTURE_DELAY")\
            .where("Origin_Airport = 'SFO'")\
                .sort(desc("DEPARTURE_DELAY"))\
                .show(5)


+-------------+---------------+
|FLIGHT_NUMBER|DEPARTURE_DELAY|
+-------------+---------------+
|         2293|           1365|
|          379|           1109|
|          338|            683|
|          338|            577|
|          658|            569|
+-------------+---------------+
only showing top 5 rows



In [19]:
#5.Which destinations tend to have delays

df6 = df.select("DESTINATION_AIRPORT","ARRIVAL_DELAY")\
        .where("ARRIVAL_DELAY > 0")\
        .sort(desc("Arrival_Delay"))\
        .show(5)

+-------------------+-------------+
|DESTINATION_AIRPORT|ARRIVAL_DELAY|
+-------------------+-------------+
|                DFW|         1971|
|                MIA|         1627|
|                LAX|         1593|
|                LAX|         1576|
|                DFW|         1554|
+-------------------+-------------+
only showing top 5 rows



##Which destinations tend to have significant delays departing from SEA

In [20]:


df7 = df.select("DESTINATION_AIRPORT","ARRIVAL_DELAY")\
        .where("Origin_AIRPORT = 'SEA'")\
        .orderBy(desc("Arrival_Delay"))\
        .show(5)

+-------------------+-------------+
|DESTINATION_AIRPORT|ARRIVAL_DELAY|
+-------------------+-------------+
|                MSP|          805|
|                CLT|          621|
|                PHX|          516|
|                CLT|          481|
|                DEN|          474|
+-------------------+-------------+
only showing top 5 rows



##Airport Ranking using Rank

In [35]:
#Airport Ranking using Rank - Count the Origin_airport and rank accordingly

df8 = df.select("ORIGIN_AIRPORT")\
         .groupby("ORIGIN_AIRPORT").count().alias('count')\
         .orderBy(desc("count"))\
         .limit(5)




In [38]:
df8.show()

+--------------+-----+
|ORIGIN_AIRPORT|count|
+--------------+-----+
|           ATL|66599|
|           ORD|52961|
|           DFW|50933|
|           LAX|38473|
|           DEN|38254|
+--------------+-----+



In [48]:
from pyspark.sql import Window
from pyspark.sql.functions import rank

df9 = df8.withColumn("rank",rank().over(Window.orderBy(desc("count"))))

In [49]:
df9.show(5)


+--------------+-----+----+
|ORIGIN_AIRPORT|count|rank|
+--------------+-----+----+
|           ATL|66599|   1|
|           ORD|52961|   2|
|           DFW|50933|   3|
|           LAX|38473|   4|
|           DEN|38254|   5|
+--------------+-----+----+



In [30]:
from pyspark.sql.functions import lit
df9 = df8.withColumn("count",lit(3000))
df9.show(5)

+--------------+-----+
|ORIGIN_AIRPORT|count|
+--------------+-----+
|           ATL| 3000|
|           ORD| 3000|
|           DFW| 3000|
|           LAX| 3000|
|           DEN| 3000|
+--------------+-----+

